In [10]:
import pandas as pd

import tensorflow as tf

# display all columns
# pd.options.display.max_columns = 
# pd.set_option('display.max_colwidth', None)

pd.options.display.max_columns = None
pd.options.display.max_colwidth = None

In [11]:
# Utils
from sklearn import preprocessing
def encode_text_index(df, name):
    le = preprocessing.LabelEncoder()
    df[name] = le.fit_transform(df[name])
    return le.classes_
# Encode a numeric column as zscores
def encode_numeric_zscore(df, name, mean=None, sd=None):
    if mean is None:
        mean = df[name].mean()

    if sd is None:
        sd = df[name].std()

    df[name] = (df[name] - mean) / sd
# Encode text values to dummy variables(i.e. [1,0,0],[0,1,0],[0,0,1] for red,green,blue)
def encode_text_dummy(df, name):
    dummies = pd.get_dummies(df[name])
    for x in dummies.columns:
        dummy_name = f"{name}-{x}"
        df[dummy_name] = dummies[x]
    df.drop(name, axis=1, inplace=True)
# Convert a Pandas dataframe to the x,y inputs that TensorFlow needs
def to_xy(df, target):
    result = []
    for x in df.columns:
        if x != target:
            result.append(x)
    # find out the type of the target column.  Is it really this hard? :(
    target_type = df[target].dtypes
    target_type = target_type[0] if hasattr(
        target_type, '__iter__') else target_type
    # Encode to int for classification, float otherwise. TensorFlow likes 32 bits.
    if target_type in (np.int64, np.int32):
        # Classification
        dummies = pd.get_dummies(df[target])
        return df[result].values.astype(np.float32), dummies.values.astype(np.float32)
    # Regression
    return df[result].values.astype(np.float32), df[[target]].values.astype(np.float32)

In [13]:
path = "/Users/nguyenquangminh/Desktop/Thesis/Dataset/UNSW-NB15/UNSW_NB15_training-set.csv"
# Pre processing
dataset = pd.read_csv(path)
dataset["attack_cat"].unique()

array(['Normal', 'Reconnaissance', 'Backdoor', 'DoS', 'Exploits',
       'Analysis', 'Fuzzers', 'Worms', 'Shellcode', 'Generic'],
      dtype=object)

In [14]:
dataset.drop(columns=["id", "label"], inplace=True)
for column in dataset.columns:
    if column in ["proto", "service", "state"]:
        encode_text_dummy(dataset, column)
    elif column == "attack_cat":
        encode_text_index(dataset, column)
    else:
        encode_numeric_zscore(dataset, column)
dataset

/var/folders/vz/vzbh0r6s7cs02cllngw126jh0000gn/T/ipykernel_72028/3650344082.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[dummy_name] = dummies[x]
/var/folders/vz/vzbh0r6s7cs02cllngw126jh0000gn/T/ipykernel_72028/3650344082.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[dummy_name] = dummies[x]
/var/folders/vz/vzbh0r6s7cs02cllngw126jh0000gn/T/ipykernel_72028/3650344082.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor perf

/var/folders/vz/vzbh0r6s7cs02cllngw126jh0000gn/T/ipykernel_72028/3650344082.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[dummy_name] = dummies[x]
/var/folders/vz/vzbh0r6s7cs02cllngw126jh0000gn/T/ipykernel_72028/3650344082.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[dummy_name] = dummies[x]
/var/folders/vz/vzbh0r6s7cs02cllngw126jh0000gn/T/ipykernel_72028/3650344082.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor perf

,dur,spkts,dpkts,sbytes,dbytes,rate,sttl,dttl,sload,dload,sloss,dloss,sinpkt,dinpkt,sjit,djit,swin,stcpb,dtcpb,dwin,tcprtt,synack,ackdat,smean,dmean,trans_depth,response_body_len,ct_srv_src,ct_state_ttl,ct_dst_ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,proto-3pc,proto-a/n,proto-aes-sp3-d,proto-any,proto-argus,proto-aris,proto-arp,proto-ax.25,proto-bbn-rcc,proto-bna,proto-br-sat-mon,proto-cbt,proto-cftp,proto-chaos,proto-compaq-peer,proto-cphb,proto-cpnx,proto-crtp,proto-crudp,proto-dcn,proto-ddp,proto-ddx,proto-dgp,proto-egp,proto-eigrp,proto-emcon,proto-encap,proto-etherip,proto-fc,proto-fire,proto-ggp,proto-gmtp,proto-gre,proto-hmp,proto-i-nlsp,proto-iatp,proto-ib,proto-idpr,proto-idpr-cmtp,proto-idrp,proto-ifmp,proto-igmp,proto-igp,proto-il,proto-ip,proto-ipcomp,proto-ipcv,proto-ipip,proto-iplt,proto-ipnip,proto-ippc,proto-ipv6,proto-ipv6-frag,proto-ipv6-no,proto-ipv6-opts,proto-ipv6-route,proto-ipx-n-ip,proto-irtp,proto-isis,proto-iso-ip,proto-iso-tp4,proto-kryptolan,proto-l2tp,proto-larp,proto-leaf-1,proto-leaf-2,proto-merit-inp,proto-mfe-nsp,proto-mhrp,proto-micp,proto-mobile,proto-mtp,proto-mux,proto-narp,proto-netblt,proto-nsfnet-igp,proto-nvp,proto-ospf,proto-pgm,proto-pim,proto-pipe,proto-pnni,proto-pri-enc,proto-prm,proto-ptp,proto-pup,proto-pvp,proto-qnx,proto-rdp,proto-rsvp,proto-rvd,proto-sat-expak,proto-sat-mon,proto-sccopmce,proto-scps,proto-sctp,proto-sdrp,proto-secure-vmtp,proto-sep,proto-skip,proto-sm,proto-smp,proto-snp,proto-sprite-rpc,proto-sps,proto-srp,proto-st2,proto-stp,proto-sun-nd,proto-swipe,proto-tcf,proto-tcp,proto-tlsp,proto-tp++,proto-trunk-1,proto-trunk-2,proto-ttp,proto-udp,proto-unas,proto-uti,proto-vines,proto-visa,proto-vmtp,proto-vrrp,proto-wb-expak,proto-wb-mon,proto-wsn,proto-xnet,proto-xns-idp,proto-xtp,proto-zero,service--,service-dhcp,service-dns,service-ftp,service-ftp-data,service-http,service-irc,service-pop3,service-radius,service-smtp,service-snmp,service-ssh,service-ssl,state-ACC,state-CLO,state-CON,state-FIN,state-INT,state-REQ,state-RST
0,-0.213726,-0.124454,-0.151815,-0.043683,-0.087368,0.057181,0.719436,-0.820390,0.643909,-0.263496,-0.073530,-0.113243,-0.122179,-0.094168,-0.112176,-0.147217,-1.047913,-0.779835,-0.776749,-1.006238,-0.482022,-0.412907,-0.484070,0.520316,-0.475368,-0.173647,-0.04191,-0.680470,0.591017,-0.563656,-0.468309,-0.450184,-0.477991,-0.090857,-0.090617,-0.203142,-0.640029,-0.644186,-0.106069,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,-0.213727,-0.124454,-0.151815,-0.036308,-0.087368,0.286563,0.719436,-0.820390,4.539323,-0.263496,-0.073530,-0.113243,-0.122179,-0.094168,-0.112176,-0.147217,-1.047913,-0.779835,-0.776749,-1.006238,-0.482022,-0.412907,-0.484070,3.556694,-0.475368,-0.173647,-0.04191,-0.680470,0.591017,-0.563656,-0.468309,-0.450184,-0.477991,-0.090857,-0.090617,-0.203142,-0.640029,-0.644186,-0.106069,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,-0.213727,-0.124454,-0.151815,-0.040351,-0.087368,0.791205,0.719436,-0.820390,4.391432,-0.263496,-0.073530,-0.113243,-0.122180,-0.094168,-0.112176,-0.147217,-1.047913,-0.779835,-0.776749,-1.006238,-0.482022,-0.412907,-0.484070,1.892203,-0.475368,-0.173647,-0.04191,-0.590301,0.591017,-0.563656,-0.468309,-0.450184,-0.390389,-0.090857,-0.090617,-0.203142,-0.640029,-0.554270,-0.106069,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [15]:
import pandas as pd
import io
import requests
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn import metrics
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.callbacks import EarlyStopping

# Break into X (predictors) & y (prediction)
x, y = to_xy(dataset,'attack_cat')

# Create a test/train split.  25% test
# Split into train/test
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.25, random_state=42)

print(x.shape)

# Create neural net
model = Sequential()
model.add(Dense(10, input_dim=x.shape[1], kernel_initializer='normal', activation='relu'))
model.add(Dense(50, input_dim=x.shape[1], kernel_initializer='normal', activation='relu'))
model.add(Dense(10, input_dim=x.shape[1], kernel_initializer='normal', activation='relu'))
model.add(Dense(1, kernel_initializer='normal'))
model.add(Dense(y.shape[1],activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
print(model.summary())
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=1, mode='auto')
model.fit(x_train,y_train,validation_data=(x_test,y_test),callbacks=[monitor],verbose=2,epochs=1000)

(82332, 190)
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (None, 10)                1910      
                                                                 
 dense_6 (Dense)             (None, 50)                550       
                                                                 
 dense_7 (Dense)             (None, 10)                510       
                                                                 
 dense_8 (Dense)             (None, 1)                 11        
                                                                 
 dense_9 (Dense)             (None, 10)                20        
                                                                 
Total params: 3001 (11.72 KB)
Trainable params: 3001 (11.72 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
Ep

In [16]:
# Measure accuracy
pred = model.predict(x_test)
pred = np.argmax(pred,axis=1)
y_eval = np.argmax(y_test,axis=1)
score = metrics.accuracy_score(y_eval, pred)
print("Validation score: {}".format(score))

644/644 [==============================] - 0s 590us/step
Validation score: 0.8049361123257057
